### Import dependencies

In [1]:
import polars_bio as pb
import pandas as pd

/home/user/.pyenv/versions/3.12.9/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:polars_bio:Creating BioSessionContext


### Usage examples

#### Usage example - calling UDAF directly in SQL

In [ ]:
a_lazyframe = pb.read_fastq("./example.fastq")
# not implemented yet
result = pb.sql("???").collect()
print(result)

#### Usage example - .fastq file

In [3]:
result = pb.base_sequence_quality("example.fastq", output_type="pandas.DataFrame")
print(result)

       pos  score
0        0      2
1        1     19
2        2     33
3        3     35
4        4     37
...    ...    ...
20195   96     35
20196   97     32
20197   98     35
20198   99     35
20199  100     33

[20200 rows x 2 columns]


#### Usage example - .csv file

In [4]:
pb.base_sequence_quality("example.csv")

pos,score
i64,i8
0,2
1,19
2,33
3,35
4,37
…,…
96,35
97,32
98,35


#### Usage example - .parquet file

In [5]:
pb.base_sequence_quality("example.parquet")

pos,score
i64,i8
0,2
1,19
2,33
3,35
4,37
…,…
96,35
97,32
98,35


#### Usage example - `polars.lazyframe.frame.LazyFrame` object

In [6]:
a_lazyframe = pb.read_fastq("./example.fastq")
print(type(a_lazyframe))
pb.base_sequence_quality(a_lazyframe)

INFO:polars_bio:Table: example registered for path: ./example.fastq


<class 'polars.lazyframe.frame.LazyFrame'>


200rows [00:00, 85580.58rows/s]


pos,score
i64,i8
0,2
1,19
2,33
3,35
4,37
…,…
96,35
97,32
98,35


#### Usage example - `polars.dataframe.frame.DataFrame` object

In [7]:
a_lazyframe = pb.read_fastq("./example.fastq")
a_dataframe = a_lazyframe.collect()
print(type(a_dataframe))
pb.base_sequence_quality(a_dataframe)

INFO:polars_bio:Table: example registered for path: ./example.fastq
200rows [00:00, 99332.24rows/s]

<class 'polars.dataframe.frame.DataFrame'>


pos,score
i64,i8
0,2
1,19
2,33
3,35
4,37
…,…
96,35
97,32
98,35


#### Usage example - `pandas.core.frame.DataFrame` object

In [8]:
a_lazyframe = pb.read_fastq("./example.fastq")
a_pandas_dataframe = a_lazyframe.collect().to_pandas()
print(type(a_pandas_dataframe))
pb.base_sequence_quality(a_pandas_dataframe)

INFO:polars_bio:Table: example registered for path: ./example.fastq
200rows [00:00, 91638.72rows/s]

<class 'pandas.core.frame.DataFrame'>


pos,score
i64,i8
0,2
1,19
2,33
3,35
4,37
…,…
96,35
97,32
98,35
